In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "import collections\n",
    "import json\n",
    "import os\n",
    "\n",
    "from absl import logging\n",
    "import tensorflow.compat.v2 as tf\n",
    "\n",
    "import tensorflow_datasets.public_api as tfds"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "\"\"\"mvtec_screws COCO STYLE DATASET.\"\"\"\n",
    "_CITATION = \"\"\"\\\n",
    "@article{NA,\n",
    "  author    = {NA},\n",
    "  title     = {MVTEC SCREWS DATASET},\n",
    "  journal   = {NA},\n",
    "  volume    = {NA},\n",
    "  year      = {NA},\n",
    "  url       = {NA},\n",
    "  archivePrefix = {NA},\n",
    "  eprint    = {NA},\n",
    "  timestamp = {NA},\n",
    "  biburl    = {NA},\n",
    "  bibsource = {NA},\n",
    "}\n",
    "\"\"\"\n",
    "\n",
    "_DESCRIPTION = \"\"\"\n",
    "Note:\n",
    " * \n",
    "\"\"\"\n",
    "\n",
    "_CONFIG_DESCRIPTION = \"\"\"\n",
    "This version contains images, bounding boxes, orientation and labels.\n",
    "\"\"\"\n",
    "\n",
    "Split = collections.namedtuple(\n",
    "    'Split', ['name', 'images', 'annotations', 'annotation_type'])\n",
    "\n",
    "\n",
    "class AnnotationType(object):\n",
    "  \"\"\"Enum of the annotation format types.\n",
    "\n",
    "  Splits are annotated with different formats.\n",
    "  \"\"\"\n",
    "  BBOXES = 'bboxes'\n",
    "  NONE = 'none'\n",
    "\n",
    "\n",
    "class MVTEC_SCREWSConfig(tfds.core.BuilderConfig):\n",
    "  \"\"\"BuilderConfig for mvtec_screwsConfig.\"\"\"\n",
    "\n",
    "  def __init__(self, splits=None, **kwargs):\n",
    "    super(MVTEC_SCREWSConfig, self).__init__(\n",
    "        version=tfds.core.Version('1.1.0'), **kwargs)\n",
    "    self.splits = splits\n",
    "\n",
    "\n",
    "\n",
    "class MVTEC_SCREWS(tfds.core.GeneratorBasedBuilder):\n",
    "  \"\"\"Base Screws dataset.\"\"\"\n",
    "\n",
    "  MANUAL_DOWNLOAD_INSTRUCTIONS = \"\"\"\n",
    "  Register into https://example.org/login to get the data. Place the `data.zip`\n",
    "  file in the `manual_dir/`.\n",
    "  \"\"\"\n",
    "\n",
    "  BUILDER_CONFIGS = [\n",
    "      MVTEC_SCREWSConfig(\n",
    "          name='mvtec_screws',\n",
    "          description=_CONFIG_DESCRIPTION.format(year=2022),\n",
    "          splits=[\n",
    "              Split(\n",
    "                  name=tfds.Split.TRAIN,\n",
    "                  images='train',\n",
    "                  annotations='annotations_trainval',\n",
    "                  annotation_type=AnnotationType.BBOXES,\n",
    "              ),\n",
    "          ],\n",
    "      ),\n",
    "  ]\n",
    "\n",
    "  def _info(self):\n",
    "    features = {\n",
    "        # Images can have variable shape\n",
    "        'image': tfds.features.Image(),\n",
    "        'image/filename': tfds.features.Text(),\n",
    "        'image/id': tf.int64,\n",
    "    }\n",
    "    # Uses original annotations\n",
    "    if True:\n",
    "      features.update({\n",
    "          'objects':\n",
    "              tfds.features.Sequence({\n",
    "                  'id': tf.int64,\n",
    "                  # Coco has unique id for each annotation. The id can be used\n",
    "                  # for mapping panoptic image to semantic segmentation label.\n",
    "                  'area': tf.int64,\n",
    "                  'bbox': tfds.features.BBoxFeature(),\n",
    "                  'phi': tf.float32,\n",
    "                  'label': tfds.features.ClassLabel(num_classes=13),\n",
    "                  'is_crowd': tf.bool,\n",
    "              }),\n",
    "      })\n",
    "    # More info could be added, like segmentation (as png mask), captions,\n",
    "    # person key-points, more metadata (original flickr url,...).\n",
    "\n",
    "    return tfds.core.DatasetInfo(\n",
    "        builder=self,\n",
    "        description=_DESCRIPTION,\n",
    "        # More info could be added, like the segmentation (as png mask),\n",
    "        # captions, person key-points. For caption encoding, it would probably\n",
    "        # be better to have a separate class CocoCaption2014 to avoid poluting\n",
    "        # the main class with builder config for each encoder.\n",
    "        features=tfds.features.FeaturesDict(features),\n",
    "        homepage='NA',\n",
    "        citation=_CITATION,\n",
    "    )\n",
    "\n",
    "  def _split_generators(self, dl_manager):\n",
    "    # data_path is a pathlib-like `Path('<manual_dir>/data.zip')`\n",
    "    archive_path = dl_manager.manual_dir / 'mvtec_screws_data.zip'\n",
    "    print(dl_manager.manual_dir)\n",
    "    # Extract the manually downloaded `data.zip`\n",
    "    extracted_path = dl_manager.extract(archive_path)\n",
    "\n",
    "    extracted_paths = dict()\n",
    "    with os.scandir(extracted_path) as file_list:\n",
    "        for f in file_list:\n",
    "            print(f.name)\n",
    "            extracted_paths[f.name] = f.path\n",
    "\n",
    "\n",
    "    splits = []\n",
    "    for split in self.builder_config.splits:\n",
    "      image_dir = extracted_paths['{}_images'.format(split.name)]\n",
    "      annotations_dir = extracted_paths['{}_annotations'.format(split.name)]\n",
    "\n",
    "      splits.append(\n",
    "          tfds.core.SplitGenerator(\n",
    "              name=split.name,\n",
    "              gen_kwargs=dict(\n",
    "                  image_dir=image_dir,\n",
    "                  annotation_dir=annotations_dir,\n",
    "                  split_name=split.images,\n",
    "                  annotation_type=split.annotation_type,\n",
    "              ),\n",
    "          ))\n",
    "    return splits\n",
    "\n",
    "  def _generate_examples(self, image_dir, annotation_dir, split_name,\n",
    "                         annotation_type):\n",
    "    \"\"\"Generate examples as dicts.\n",
    "\n",
    "    Args:\n",
    "      image_dir: `str`, directory containing the images\n",
    "      annotation_dir: `str`, directory containing annotations\n",
    "      split_name: `str`, <split_name><year> (ex: train2014, val2017)\n",
    "      annotation_type: `AnnotationType`, the annotation format (NONE, BBOXES)\n",
    "\n",
    "    Yields:\n",
    "      example key and data\n",
    "    \"\"\"\n",
    "\n",
    "    if annotation_type == AnnotationType.BBOXES:\n",
    "      instance_filename = 'instances_{}.json'\n",
    "    elif annotation_type == AnnotationType.NONE:  # No annotation for test sets\n",
    "      instance_filename = 'image_info_{}.json'\n",
    "\n",
    "    # Load the annotations (label names, images metadata,...)\n",
    "    instance_path = os.path.join(\n",
    "        annotation_dir,\n",
    "        'annotations',\n",
    "        instance_filename.format(split_name),\n",
    "    )\n",
    "    coco_annotation = ANNOTATION_CLS[annotation_type](instance_path)\n",
    "    # Each category is a dict:\n",
    "    # {\n",
    "    #    'id': 51,  # From 1-91, some entry missing\n",
    "    #    'name': 'bowl',\n",
    "    #    'supercategory': 'kitchen',\n",
    "    # }\n",
    "    categories = coco_annotation.categories\n",
    "    # Each image is a dict:\n",
    "    # {\n",
    "    #     'id': 262145,\n",
    "    #     'file_name': 'COCO_train2017_000000262145.jpg'\n",
    "    #     'flickr_url': 'http://farm8.staticflickr.com/7187/xyz.jpg',\n",
    "    #     'coco_url': 'http://images.cocodataset.org/train2017/xyz.jpg',\n",
    "    #     'license': 2,\n",
    "    #     'date_captured': '2013-11-20 02:07:55',\n",
    "    #     'height': 427,\n",
    "    #     'width': 640,\n",
    "    # }\n",
    "    images = coco_annotation.images\n",
    "\n",
    "    # TODO(b/121375022): ClassLabel names should also contains 'id' and\n",
    "    # and 'supercategory' (in addition to 'name')\n",
    "    # Warning: As Coco only use 80 out of the 91 labels, the c['id'] and\n",
    "    # dataset names ids won't match.\n",
    "    if True:\n",
    "      objects_key = 'objects'\n",
    "    self.info.features[objects_key]['label'].names = [\n",
    "        c['name'] for c in categories\n",
    "    ]\n",
    "    # TODO(b/121375022): Conversion should be done by ClassLabel\n",
    "    categories_id2name = {c['id']: c['name'] for c in categories}\n",
    "\n",
    "    # Iterate over all images\n",
    "    annotation_skipped = 0\n",
    "    for image_info in sorted(images, key=lambda x: x['id']):\n",
    "      if annotation_type == AnnotationType.BBOXES:\n",
    "        # Each instance annotation is a dict:\n",
    "        # {\n",
    "        #     'iscrowd': 0,\n",
    "        #     'bbox': [116.95, 305.86, 285.3, 266.03],\n",
    "        #     'image_id': 480023,\n",
    "        #     'segmentation': [[312.29, 562.89, 402.25, ...]],\n",
    "        #     'category_id': 58,\n",
    "        #     'area': 54652.9556,\n",
    "        #     'id': 86,\n",
    "        # }\n",
    "        instances = coco_annotation.get_annotations(img_id=image_info['id'])\n",
    "\n",
    "      else:\n",
    "        instances = []  # No annotations\n",
    "\n",
    "      if not instances:\n",
    "        annotation_skipped += 1\n",
    "\n",
    "      def build_bbox(x, y, width, height):\n",
    "        # pylint: disable=cell-var-from-loop\n",
    "        # build_bbox is only used within the loop so it is ok to use image_info\n",
    "        return tfds.features.BBox(\n",
    "            ymin=(y - y*.5)/ image_info['height'],\n",
    "            xmin=(x - x*.5)/ image_info['width'],\n",
    "            ymax=((y - y*.5) + height) / image_info['height'],\n",
    "            xmax=((x - x*.5) + width) / image_info['width'],\n",
    "            #ymin=y,\n",
    "            #xmin=x,\n",
    "            #ymax=y+height,\n",
    "            #xmax=x+width,\n",
    "        )\n",
    "        # pylint: enable=cell-var-from-loop\n",
    "\n",
    "      example = {\n",
    "          'image': os.path.join(image_dir, split_name, image_info['file_name']),\n",
    "          'image/filename': image_info['file_name'],\n",
    "          'image/id': image_info['id'],\n",
    "          objects_key: [{   # pylint: disable=g-complex-comprehension\n",
    "              'id': instance['id'],\n",
    "              'area': instance['area'],\n",
    "              'bbox': build_bbox(instance['bbox'][0],instance['bbox'][1],instance['bbox'][2], instance['bbox'][3]),\n",
    "              'phi': instance['bbox'][4],\n",
    "              'label': categories_id2name[instance['category_id']],\n",
    "              'is_crowd': bool(instance['is_crowd']),\n",
    "          } for instance in instances]\n",
    "      }\n",
    "\n",
    "      yield image_info['file_name'], example\n",
    "\n",
    "    logging.info(\n",
    "        '%d/%d images do not contains any annotations',\n",
    "        annotation_skipped,\n",
    "        len(images),\n",
    "    )\n",
    "\n",
    "\n",
    "class CocoAnnotation(object):\n",
    "  \"\"\"Coco annotation helper class.\"\"\"\n",
    "\n",
    "  def __init__(self, annotation_path):\n",
    "    with tf.io.gfile.GFile(annotation_path) as f:\n",
    "      data = json.load(f)\n",
    "    self._data = data\n",
    "\n",
    "  @property\n",
    "  def categories(self):\n",
    "    \"\"\"Return the category dicts, as sorted in the file.\"\"\"\n",
    "    return self._data['categories']\n",
    "\n",
    "  @property\n",
    "  def images(self):\n",
    "    \"\"\"Return the image dicts, as sorted in the file.\"\"\"\n",
    "    return self._data['images']\n",
    "\n",
    "  def get_annotations(self, img_id):\n",
    "    \"\"\"Return all annotations associated with the image id string.\"\"\"\n",
    "    raise NotImplementedError  # AnotationType.NONE don't have annotations\n",
    "\n",
    "\n",
    "class CocoAnnotationBBoxes(CocoAnnotation):\n",
    "  \"\"\"Coco annotation helper class.\"\"\"\n",
    "\n",
    "  def __init__(self, annotation_path):\n",
    "    super(CocoAnnotationBBoxes, self).__init__(annotation_path)\n",
    "\n",
    "    img_id2annotations = collections.defaultdict(list)\n",
    "    for a in self._data['annotations']:\n",
    "      img_id2annotations[a['image_id']].append(a)\n",
    "    self._img_id2annotations = {\n",
    "        k: list(sorted(v, key=lambda a: a['id']))\n",
    "        for k, v in img_id2annotations.items()\n",
    "    }\n",
    "\n",
    "  def get_annotations(self, img_id):\n",
    "    \"\"\"Return all annotations associated with the image id  string.\"\"\"\n",
    "    # Some images don't have any annotations. Return empty list instead.\n",
    "    return self._img_id2annotations.get(img_id, [])\n",
    "\n",
    "\n",
    "class CocoAnnotationPanoptic(CocoAnnotation):\n",
    "  \"\"\"Coco annotation helper class.\"\"\"\n",
    "\n",
    "  def __init__(self, annotation_path):\n",
    "    super(CocoAnnotationPanoptic, self).__init__(annotation_path)\n",
    "    self._img_id2annotations = {\n",
    "        a['image_id']: a for a in self._data['annotations']\n",
    "    }\n",
    "\n",
    "  def get_annotations(self, img_id):\n",
    "    \"\"\"Return all annotations associated with the image id string.\"\"\"\n",
    "    return self._img_id2annotations[img_id]\n",
    "\n",
    "ANNOTATION_CLS = {\n",
    "    AnnotationType.NONE: CocoAnnotation,\n",
    "    AnnotationType.BBOXES: CocoAnnotationBBoxes,\n",
    "}"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "/home/amarchia/repos/CV22Project\n"
     ]
    }
   ],
   "source": [
    "import os\n",
    "print(os.getcwd())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "2022-03-18 22:38:57.125442: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-03-18 22:38:57.126122: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-03-18 22:38:57.132890: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-03-18 22:38:57.133569: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-03-18 22:38:57.134249: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-03-18 22:38:57.134792: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-03-18 22:38:57.136227: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA\n",
      "To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.\n",
      "2022-03-18 22:38:57.321711: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-03-18 22:38:57.322541: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-03-18 22:38:57.323214: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-03-18 22:38:57.323952: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-03-18 22:38:57.324612: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-03-18 22:38:57.325312: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-03-18 22:38:58.059259: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-03-18 22:38:58.059823: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-03-18 22:38:58.060322: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-03-18 22:38:58.060904: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-03-18 22:38:58.061395: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-03-18 22:38:58.061975: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7398 MB memory:  -> device: 0, name: GeForce GTX 1070, pci bus id: 0000:0d:00.0, compute capability: 6.1\n",
      "2022-03-18 22:38:58.062460: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero\n",
      "2022-03-18 22:38:58.062910: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 5385 MB memory:  -> device: 1, name: GeForce GTX 1060 6GB, pci bus id: 0000:0c:00.0, compute capability: 6.1\n"
     ]
    }
   ],
   "source": [
    "if __name__ == \"__main__\":\n",
    "    ds = tfds.load(\"mvtec_screws\")"
   ]
  }
 ],
 "metadata": {
  "interpreter": {
   "hash": "61c63460f299b72050a017299847de40e48391aa12573d6685850838867f4476"
  },
  "kernelspec": {
   "display_name": "TensorFlow Testing",
   "language": "python",
   "name": "tf-testing"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}